#  DATA VISUALIZATION PROJECT - ALL ATHLETICS COMPETITIONS FROM 2010 TO 2021

## IMPORT LIBRARIES 

In [1]:
!pip install psycopg2

In [2]:
!pip install -U simplejson

In [20]:
!pip3 install selenium
!pip3 install webdriver-manager

In [3]:

import json
import re
import time
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import psycopg2
from psycopg2.errors import UniqueViolation
from psycopg2.errors import SyntaxError
from sqlalchemy import create_engine
import ssl
from simplejson import JSONDecodeError
from tqdm import tqdm
from requests.exceptions import MissingSchema

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains




## INITIAL PARAMETERS FOR PROCESSING DATA 

In [4]:
year= 2021 # ALL FUNCTIONS ARE BASED ON YEAR THAT WILL BE PROCESSED, SO ALWAYS CHECK THIS BEFORE EXECUTING SOME CODE BELLOW
token_wheter = open('apikey.txt', 'r').read()
user_db = open('user_db.txt', 'r').read()
pass_db = open('pass_db.txt', 'r').read()

## FUNCTIONS USED BY CODE

In [5]:
def getcomp(year, pag=1):
    '''
    year is the year when competition was done

    Its return is a object soup. It´s a kind of html conteiner based an answer from server
    or can be a integer -1 when there is no information for that year


    '''  
    ssl._create_default_https_context = ssl._create_unverified_context
    get_html = requests.get(f'https://worldathletics.org/records/competition-performance-rankings?type=2&year={year}&sortBy=score&page={pag}')
    html = get_html.content
    soup = BeautifulSoup(html)
    list_results = soup.find_all('h2', attrs = {'class' : 'no-results'})
    if len(list_results) > 0:
        return -1
    return [soup, pag]


def atualiza_prova_compet(cursor, cod_prova, cod_competicao, score_prova_competicao,data_prova_competicao):
    
    '''
    This function creates e correlates proves and competitions
    '''  
    try: 
        print(f'Inserindo competicao {cod_competicao} e prova {cod_prova}')
        cursor.execute(f"INSERT INTO tb_prova_competicao (cod_prova, cod_competicao, score_prova_competicao,data_prova_competicao)" \
                       f"VALUES ({cod_prova}, {cod_competicao[0]}, {score_prova_competicao},'{data_prova_competicao}')")

    except UniqueViolation:
        print ("prova ja existe na competicao")
        
                    
def atualiza_atleta_prova_competicao(cursor,cod_prova, cod_competicao, lista_tb_atleta_competicao):
    '''
    This function find each atletes for each  prove and insert it on prove


    '''  
    cursor.execute(f"select cod_prova_competicao from tb_prova_competicao where cod_prova = {cod_prova} and cod_competicao = {cod_competicao[0]}")
    cod_prova_competicao = cursor.fetchone()[0] 
    for x in lista_tb_atleta_competicao:
        try:
            print(f'Inserindo atleta {x[0]}')
            cursor.execute(f"INSERT INTO tb_atleta_prova_competicao (cod_atleta, cod_prova_competicao, result, score)" \
                         f"VALUES ({x[0]}, {cod_prova_competicao}, '{x[1]}', {x[2]})")
        except UniqueViolation:
            print ("atleta ja existe na prova")
            
            
            
            
def atualiza_atleta(cursor,cod_atleta, lista_tb_atleta_competicao):
    '''
    This function insert atlete code in table athete
    '''  
    for x in lista_tb_atleta_competicao:
        try:
            print(f'Inserindo atleta na tb_atleta {x[0]}')
            cursor.execute(f"INSERT INTO tb_atleta (cod_atleta)" \
                         f"VALUES ({x[0]})")
        except UniqueViolation:
            print ("atleta ja existe na tb_atleta")
            
            
            
def update_atleta(cursor,cod_atleta,url_atleta,pais_atleta,nome_atleta):
    
    '''
    This function update  atlete information  on table athete
    '''  
   
        
    cursor.execute(f"UPDATE tb_atleta SET url_atleta = '{url_atleta}', " \
                         f"pais_atleta = '{pais_atleta}', nome_atleta = '{nome_atleta}' WHERE cod_atleta = {cod_atleta}")
    print (f"atleta {cod_atleta} atualizado")
            
def update_wheter(cursor,data_prova_competicao, pais_competicao, cidade_competicao, json_resp):
    '''
    This function update  weather information to every day thats occurs proves
    '''
    try:
        cursor.execute(f"INSERT INTO tb_wheter (data_prova_competicao, pais_competicao, cidade_competicao, info_wheter)" \
                         f"VALUES ('{data_prova_competicao}', '{pais_competicao}', '{cidade_competicao}', '{json_resp}')")
    except UniqueViolation:
            print ("condições climaticas já existem na base")
            
            
            
def connect_db():
    '''
    This function connect data base
    '''  
   
    import psycopg2
    from psycopg2.errors import UniqueViolation
    from psycopg2.errors import SyntaxError
    connection = psycopg2.connect(user=user_db,
                                  password=pass_db,
                                  host="127.0.0.1",
                                  port="5432",
                                  database="postgres",
                                  options="-c search_path=db_atl")


    connection.autocommit = True
    return connection.cursor()




def validate(date_text):
    from datetime import datetime
    
    try:
        datetime.strptime(data_formato, '%d-%b-%Y')
        return True
    except ValueError:
        return False


## Insertions on tb_competicao

In [ ]:
cursor = connect_db()
contador = 0
list_results = []
soup = getcomp(year,1)[0]
pagina = getcomp(year,1)[1]   
last_page = int(soup.find_all('a', attrs = {'class' : 'btn--pagination btn--pag-last pag-control'})[0]['data-page'])
list_competicao = soup.find_all('tbody')[0].find_all('tr')

if last_page > 0:
    for pagina_int in range(last_page):
        pagina_int += 1
        soup = getcomp(year,pagina_int)[0]
        pagina = getcomp(year,pagina_int)[1]   
        list_competicao = soup.find_all('tbody')[0].find_all('tr')
    
        time.sleep(5)

        for line in range(len(list_competicao)-1):
            try:
                cod_competicao = re.findall(r'\d+',list_competicao[line]['data-href'])[0]
                place = list_competicao[line].find_all('td')[0].text.strip()
                #  competition = list_competicao[line].find_all('td')[1].text.strip()
            
                tamanho = len(list_competicao[line].find_all('td')[1].text.strip().split(","))
            
                lista_nome_competicao = list_competicao[line].find_all('td')[1].text.strip().split(",")
                nome_competicao = " ".join(re.findall("[a-zA-Z]+", lista_nome_competicao[0].strip()))
                cidade_competicao =  " ".join(re.findall("[a-zA-Z]+", lista_nome_competicao[len(lista_nome_competicao)-1].strip()))
                   
         
                country = list_competicao[line].find_all('td')[2].text.strip()
                Start_date = list_competicao[line].find_all('td')[3].text.strip()
                end_date = list_competicao[line].find_all('td')[4].text.strip()
                part_score = list_competicao[line].find_all('td')[5].text.strip()
                ps_place = list_competicao[line].find_all('td')[6].text.strip()
                result_score = list_competicao[line].find_all('td')[7].text.strip()
                rs_place = list_competicao[line].find_all('td')[8].text.strip()
       
                if pagina == 1:
                   comp_score = list_competicao[line].find_all('td')[10].text.strip()
                else:
                    comp_score = list_competicao[line].find_all('td')[9].text.strip()
             
                list_results.append((cod_competicao, nome_competicao, comp_score, part_score, result_score, Start_date, end_date,
                                  cidade_competicao, country, ps_place, rs_place ))
            
            
                sql_update = f"INSERT INTO tb_competicao (cod_competicao, nome_competição, score_competicao," \
                    f"score_participacao_competicao, score_result_competicao, data_inicial_competicao," \
                    f"data_final_competicao, cidade_competicao, pais_competicao," \
                    f"ps_place_competicao, rs_place_competicao)" \
                    f" VALUES ({cod_competicao},'{nome_competicao}','{comp_score}','{part_score}','{result_score}','{Start_date}','{end_date}', "\
                    f"'{cidade_competicao}','{country}','{ps_place}','{rs_place}')"
                contador += 1  
                print (f'Executando Query {contador} executada')
                cursor.execute(sql_update)
        
           # except UniqueViolation:
           #     print("Registro encontrado")
           #     continue     ''' 
            
            except (Exception, psycopg2.DatabaseError) as error:
                print("Error in transction Reverting all other operations of a transction ", error)
    
cursor.close()
print("Transaction completed successfully ")


    
     

## Insertions on other tables

In [ ]:
cursor = connect_db()

cursor.execute(f"SELECT cod_competicao  FROM tb_competicao WHERE date_part('year', data_final_competicao) = {year};")
               
list_cod_comp = cursor.fetchall()   

ssl._create_default_https_context = ssl._create_unverified_context

lista_full = []

for compt in list_cod_comp:
    get_html = requests.get(f'https://worldathletics.org/records/competition-performance-rankings/ranking/{compt[0]}')
    print (f'https://worldathletics.org/records/competition-performance-rankings/ranking/{compt[0]}')
    html = get_html.content
    soup = BeautifulSoup(html)
    list_results = soup.find_all('div', attrs = {'class' : 'cpr-result-score'})

    list_competicao = soup.find_all('tbody')[0].find_all('tr')

    for i in list_results:
        try:
            prova = i.find_all('h3')[0].text.split("'s")
            print (f'Prova {prova[0]} na modalidade {prova[1]}')
            score_prova =i.find_all('p')[0].text.split(":")
            cursor.execute(f"INSERT INTO tb_prova (nome_prova, genero_prova) VALUES ('{prova[1]}', '{prova[0]}')")
            
            cursor.execute(f"SELECT cod_prova  FROM tb_prova WHERE nome_prova='{prova[1]}' and genero_prova='{prova[0]}';")
               
            cod_prova = cursor.fetchone()        
            print (f'codigo  da prova {cod_prova[0]}')         
            lista_tb_atleta_competicao = []
            for x in i.find_all('tbody')[0].find_all('tr'):
                score = x.find_all('td')[0].text
                result = x.find_all('td')[1].text
                date = x.find_all('td')[2].text
                cod_atleta = re.findall(r'\d+', x.find_all('td')[3].find_all('span')[0]['data-href'])[0]
                lista_full.append((cod_prova[0],score_prova[1].strip(), score.strip(), result.strip(), date.strip(), cod_atleta.strip()))    
                lista_tb_atleta_competicao.append([cod_atleta,result,score])
            atualiza_prova_compet(cursor, cod_prova[0],compt,score_prova[1], date.strip())
            atualiza_atleta(cursor,cod_atleta, lista_tb_atleta_competicao)
            atualiza_atleta_prova_competicao(cursor,cod_prova[0],compt,lista_tb_atleta_competicao)
        
        except IndexError:
            break
        except UniqueViolation:
            print ("Prova ja existe")
            cursor.execute(f"SELECT cod_prova  FROM tb_prova WHERE nome_prova='{prova[1]}' and genero_prova='{prova[0]}';")
            cod_prova = cursor.fetchone()        
            print (f'codigo  da prova {cod_prova[0]}')         
            lista_tb_atleta_competicao = []
            for x in i.find_all('tbody')[0].find_all('tr'):
                try:
                    score = x.find_all('td')[0].text
                    result = x.find_all('td')[1].text
                    date = x.find_all('td')[2].text
                    cod_atleta = re.findall(r'\d+', x.find_all('td')[3].find_all('span')[0]['data-href'])[0]
                    lista_full.append((cod_prova[0],score_prova[1].strip(), score.strip(), result.strip(), date.strip(), cod_atleta.strip()))   
                    lista_tb_atleta_competicao.append([cod_atleta,result,score])
                except IndexError:
                    break
            
            atualiza_prova_compet(cursor, cod_prova[0],compt,score_prova[1], date.strip())
            atualiza_atleta(cursor,cod_atleta, lista_tb_atleta_competicao)
            atualiza_atleta_prova_competicao(cursor,cod_prova[0],compt,lista_tb_atleta_competicao)
            continue
cursor.close()
print("Transaction completed successfully ")

## Updating climate conditions 

In [ ]:
cursor = connect_db()

ssl._create_default_https_context = ssl._create_unverified_context
count_query = 0

cursor.execute(f"select distinct * from cidades where  date_part('year', data_prova_competicao) ={year}")

             
list_data = cursor.fetchall()   


for i in list_data:
    cidade_competicao = [i][0][0]
    pais_competicao = [i][0][1]
    data_prova_competicao = [i][0][2]
    cursor.execute(f"select count (*) from tb_wheter where  cidade_competicao like '{cidade_competicao}' and "\
               f"pais_competicao like '{pais_competicao}' and data_prova_competicao = '{data_prova_competicao }'")
    list_find_reg = cursor.fetchall()   

    if list_find_reg[0][0] != 1:
        try:
alCrossingWebServices/rest/services/timeline/{cidade_competicao}%2C%20{pais_competicao}/{data_prova_competicao}/{data_prova_competicao}?            resp_json = requests.get(f'https://weather.visualcrossing.com/VisuunitGroup=metric&include=days&key={token_wheter}&contentType=json')
            if resp_json.status_code == 401:
                print (f"Vixe, chegou no limite de hoje, foram {count_query} queries")
                break
            else:
                count_query += 1
                update_wheter(cursor, data_prova_competicao, pais_competicao, cidade_competicao, json.dumps(resp_json.json()))
        except  JSONDecodeError:
                print (f"Erro na cidade {cidade_competicao} pais {pais_competicao}")
                resp_json = requests.get(f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{pais_competicao}/{data_prova_competicao}/{data_prova_competicao}?unitGroup=metric&include=days&key={token_wheter}&contentType=json')
                count_query += 1
                update_wheter(cursor, data_prova_competicao, pais_competicao, cidade_competicao, json.dumps(resp_json.json()))
               
                continue
                
        except SyntaxError:
            try:
                print (f"Erro na cidade {cidade_competicao} pais {pais_competicao}")
                resp_json = requests.get(f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{pais_competicao}/{data_prova_competicao}/{data_prova_competicao}?unitGroup=metric&include=days&key={token_wheter}&contentType=json')
                count_query += 1
                update_wheter(cursor, data_prova_competicao, pais_competicao, cidade_competicao, json.dumps(resp_json.json()))
            except SyntaxError:  
                continue
cursor.close()    
print ("fim do processamento")




## Updating athletes information 

In [ ]:

cursor = connect_db()

cursor.execute(f"SELECT cod_competicao  FROM tb_competicao WHERE date_part('year', data_final_competicao) = {year};")
               
list_cod_comp = cursor.fetchall()   

ssl._create_default_https_context = ssl._create_unverified_context

lista_full = []

for compt in list_cod_comp:
    get_html = requests.get(f'https://worldathletics.org/records/competition-performance-rankings/ranking/{compt[0]}')
    print (f'https://worldathletics.org/records/competition-performance-rankings/ranking/{compt[0]}')
    html = get_html.content
    soup = BeautifulSoup(html)
    list_results = soup.find_all('div', attrs = {'class' : 'cpr-result-score'})

    list_competicao = soup.find_all('tbody')[0].find_all('tr')
    
   
    lista_tb_atleta_competicao = []   
    

    for i in list_results:
        
       
            
            for x in i.find_all('tbody')[0].find_all('tr'):
                try:
                        cod_atleta = re.findall(r'\d+', x.find_all('td')[3].find_all('span')[0]['data-href'])[0]
                        uri_atleta = x.find_all('td')[3].find_all('span')[0]['data-href']
                        url_atleta =  'https://worldathletics.org' + uri_atleta
                        pais_atleta =  uri_atleta.split('/')[2]
                        nome_atleta =  uri_atleta.split('/')[3]
                        if cod_atleta in lista_tb_atleta_competicao:
                            print ("ja atualizou")
                        else:
                            update_atleta(cursor, cod_atleta,url_atleta,pais_atleta,nome_atleta)
                            lista_tb_atleta_competicao.append(cod_atleta)
                               
                except IndexError:
                    break
                    
            
cursor.close()
print("Transaction completed successfully ")

## Update atlhetes information on competition

In [ ]:
cursor = connect_db()

cursor.execute(f"select distinct AT.cod_atleta, AT.url_atleta from db_atl.tb_competicao as T, db_atl.tb_prova_competicao as C," \
               f"db_atl.tb_prova as P, tb_atleta_prova_competicao as A, tb_atleta as AT " \
               f" where  T.cod_competicao = C.cod_competicao and C.cod_prova_competicao = A.cod_prova_competicao and AT.nascimento_atleta IS NULL and " \
               f"C.cod_prova = P.cod_prova and P.nome_prova like '_100 Metres_%'  and AT.cod_atleta = A.cod_atleta and C.data_prova_competicao >= '2010-01-01 00:00:00';" )
       
       
list_cod_url = cursor.fetchall()   

ssl._create_default_https_context = ssl._create_unverified_context

print (f'atualizando {len(list_cod_url)} atletas' )

for compt in list_cod_url:
    try:
        get_html = requests.get(compt[1])
        print (compt[1])
        html = get_html.content
        soup = BeautifulSoup(html)
        date_born = soup.find_all('div', attrs = {'class' : 'profileBasicInfo_statValue__IXJTW'})[1].text.strip()
        idade = soup.find_all('div', attrs = {'class' : 'profileBasicInfo_statValue__IXJTW'})[3].text.strip()
        lista_dados = soup.find_all('div', attrs = {'class' : 'profileBasicInfo_statValue__IXJTW'})
        if  len(lista_dados) == 4:
            print ("idade eh ", idade)
        data_formato = date_born.replace(" ", "-")
        if  validate(data_formato):
            print (compt[1], compt[0], data_formato)
           # cursor.execute(f"UPDATE tb_atleta SET nascimento_atleta = '{data_formato}' " \
          #              f"WHERE cod_atleta = {compt[0]}")
        else:
            print ("sem data de nascimnto")
            print (compt[1], compt[0], data_formato)
    except IndexError:
        continue
    except MissingSchema:
        continue

cursor.close()
print("Transaction completed successfully ")


In [36]:
cursor = connect_db()

cursor.execute(f"SELECT data_prova_competicao, cod_atleta, url_atleta  from db_atl.db_provas_100 order by cod_atleta")           
list_cod_url = cursor.fetchall()   

ssl._create_default_https_context = ssl._create_unverified_context

print (f'atualizando {len(list_cod_url)} atletas' )

for compt in list_cod_url:
    try:
        get_html = requests.get(compt[2])
        print (compt[2])
        html = get_html.content
        soup = BeautifulSoup(html)
        lista_dados = soup.find_all('div', attrs = {'class' : 'profileBasicInfo_statValue__IXJTW'})
        
        
        # clicar em results com selenium
                 #<div class="profileStatistics_tab__1Blal">Results</div>
        
        # selecionar o ano data_prova_competicao  selenium
        '''  <select id="resultsYearSelect">
                  <option value="2021">2021</option>
                  <option value="2020">2020</option>
                  <option value="2019">2019</option>
                  <option value="2018">2018</option>
                  <option value="2017">2017</option>
                  <option value="2016">2016</option>
                  <option value="2015">2015</option>
                  <option value="2014">2014</option>
                  <option value="2013">2013</option>
                  <option value="2012">2012</option>
                  <option value="2011">2011</option>
                  <option value="2010">2010</option>
                  <option value="2009">2009</option>
                  <option value="2008">2008</option>
                  <option value="2007">2007</option>
                  </select>'''
        # ler tabela dos resultados 
                # <tbody class="profileStatistics_tableBody__1w5O9">     
        
        # guardar numa tabela nova 
                   
         
        
        if  len(lista_dados) == 4:
            idade = soup.find_all('div', attrs = {'class' : 'profileBasicInfo_statValue__IXJTW'})[3].text.strip()
            print ("idade eh ", idade)
            list_result = soup.find('div')[0].find_all('tr')   
            
            print (list_result)
            # Fazer aqui ano atual - idade 
            # inserir na base
         
        else:
            print ("sem idade")
            # inserir NaN???
            
         
       
    except IndexError:
        continue
    except MissingSchema:
        continue

cursor.close()
print("Transaction completed successfully ")


atualizando 66935 atletas
https://worldathletics.org/athletes/spain/luis-wee-palacios-014164681
idade eh  34
https://worldathletics.org/athletes/spain/luis-wee-palacios-014164681
idade eh  34
https://worldathletics.org/athletes/spain/javier-sanz-aleman-014165016
idade eh  32
https://worldathletics.org/athletes/spain/javier-sanz-aleman-014165016
idade eh  32
https://worldathletics.org/athletes/spain/javier-sanz-aleman-014165016
idade eh  32
https://worldathletics.org/athletes/spain/javier-sanz-aleman-014165016
idade eh  32
https://worldathletics.org/athletes/spain/javier-sanz-aleman-014165016
idade eh  32
https://worldathletics.org/athletes/spain/javier-sanz-aleman-014165016
idade eh  32
https://worldathletics.org/athletes/spain/javier-sanz-aleman-014165016
idade eh  32


KeyboardInterrupt: 

In [37]:
print (list_result)

[<div class="profileStatistics_contentInner__3to5y"></div>, <div class="profileStatistics_contentInner__3to5y"></div>]


<tbody class="profileStatistics_tableBody__1w5O9"><tr>
    
    
    
    

In [ ]:
list_competicao = soup.find_all('tbody')[0].find_all('tr')

if last_page > 0:
    for pagina_int in range(last_page):
        pagina_int += 1
        soup = getcomp(year,pagina_int)[0]
        pagina = getcomp(year,pagina_int)[1]   
        list_competicao = soup.find_all('tbody')[0].find_all('tr')
    
        time.sleep(5)

        for line in range(len(list_competicao)-1):
            try:
                cod_competicao = re.findall(r'\d+',list_competicao[line]['data-href'])[0]
                place = list_competicao[line].find_all('td')[0].text.strip()
                #  competition = list_competicao[line].find_all('td')[1].text.strip()
            
                tamanho = len(list_competicao[line].find_all('td')[1].text.strip().split(","))
            
                lista_nome_competicao = list_competicao[line].find_all('td')[1].text.strip().split(",")
                nome_competicao = " ".join(re.findall("[a-zA-Z]+", lista_nome_competicao[0].strip()))
                cidade_competicao =  " ".join(re.findall("[a-zA-Z]+", lista_nome_competicao[len(lista_nome_competicao)-1].strip()))
                   
         
                country = list_competicao[line].find_all('td')[2].text.strip()
                Start_date = list_competicao[line].find_all('td')[3].text.strip()
                end_date = list_competicao[line].find_all('td')[4].text.strip()
                part_score = list_competicao[line].find_all('td')[5].text.strip()
                ps_place = list_competicao[line].find_all('td')[6].text.strip()
                result_score = list_competicao[line].find_all('td')[7].text.strip()
                rs_place = list_competicao[line].find_all('td')[8].text.strip()
       
                if pagina == 1:
                   comp_score = list_competicao[line].find_all('td')[10].text.strip()
                else:
                    comp_score = list_competicao[line].find_all('td')[9].text.strip()
             
                list_results.append((cod_competicao, nome_competicao, comp_score, part_score, result_score, Start_date, end_date,
                                  cidade_competicao, country, ps_place, rs_place ))
            
            
                sql_update = f"INSERT INTO tb_competicao (cod_competicao, nome_competição, score_competicao," \
                    f"score_participacao_competicao, score_result_competicao, data_inicial_competicao," \
                    f"data_final_competicao, cidade_competicao, pais_competicao," \
                    f"ps_place_competicao, rs_place_competicao)" \
                    f" VALUES ({cod_competicao},'{nome_competicao}','{comp_score}','{part_score}','{result_score}','{Start_date}','{end_date}', "\
                    f"'{cidade_competicao}','{country}','{ps_place}','{rs_place}')"
                contador += 1  
                print (f'Executando Query {contador} executada')
                cursor.execute(sql_update)
        
           # except UniqueViolation:
           #     print("Registro encontrado")
           #     continue     ''' 
            
            except (Exception, psycopg2.DatabaseError) as error:
                print("Error in transction Reverting all other operations of a transction ", error)
    
cursor.close()
print("Transaction completed successfully ")
